In [2]:
import pandas as pd
import numpy as np
import ast

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.abspath(''))))

from yumspeak_ml.params import *
from yumspeak_ml.ml_logic.data import clean_data

In [2]:
len(CATS_TO_REMOVE)

343

In [3]:
restaurant_cats = ['Restaurant Poise',
 'Revolver',
 'Rod Dee Thai Cafe',
 'Dickson Nasi Lemak',
 'SKAI Restaurant',
 'SUSHISAMBA Singapore',
 'Saap Saap Thai',
 'Salt & Pepper Pte Ltd',
 'San Laksa Steamboat Pte Ltd',
 'San Shu Gong Private Dining 三叔公私房菜',
 'Sanook Kitchen - West Gate',
 'Sanook Kitchen - City Square',
 'Sakari Sushi',
 'Warung Selera Masakan Kampung',
 'Selera Utama Kampung Plentong Baru (Sup Power)',
 'Selera Kampung',
 'Restoran Selera Sambal',
 'Medan Selera Kampung Tengah Gedok',
 'Restoran Selera Kampung',
 'Restoran Selera Kampung (FAM MAJU)',
 'SHUNFENG 顺风川渝私房菜',
 '蜀都丰 SO DO FUN @ MIDTOWN HOUSE',
 'Hibiscus Tree (芙蓉树下) Millenia Walk',
 '天府李川菜 DAINTY SICHUAN',
 '辣不辣 Labula BBQ Sichuan Restaurant @ Hougang Village',
 'Fukurou Kitchen',
 'Sin Manbok Korean Restaurant 신만복',
 'Sinpopo Brand Restaurant @ TANGS',
 'Siri House Dempsey',
 'Sky View Pavilion',
 'Soi 19 Thai Wanton Mee Telok Ayer',
 'Soon Huat (JB) Bak Kut Teh',
 'Steamboat Charcoal Claypot Prawn @Toa Payoh',
 'Crab Wang Imperial Dining',
 'Sum Dim Sum (Bedok)',
 'Sum Dim Sum (Punggol)',
 'Summer Hill',
 'Tamba',
 'Riverwalk Tandoor',
 'Ashoka Tandoor Family Restaurant',
 'Tandoor Lounge | Indian Restaurant Punggol',
 'Tangled',
 'Tapas,24 Singapore',
 'Sap Ubon Thai Cuisine (Homebased)',
 'Thaitanium',
 'The Banana Leaf Apolo',
 'The Curry Wok (TCW)',
 'The French Ladle',
 'The Guild',
 'The Kongsee',
 'The NCO Club',
 'The Palace Korean Restaurant',
 'Simply Peranakan Cuisine',
 'The Tavern Restaurant',
 'Thevar',
 'Three Blind Pigs',
 'Tian Yu Tian 天与天 Fish Head Steamboat',
 'Tian Tian You Yu 天天友魚',
 'Tian Wai Tian Fish Head Steamboat(天外天鱼头炉）Veerasamy Road',
 'Tiffin Room',
 "Tilly's by Butcher Boy",
 'Tok Panjang@Bt. Timah',
 'Toma By The River',
 'Town Restaurant',
 'Istanbul Corner SG',
 'Eden Restaurant',
 'The Mask Restaurant and Bar',
 'UNCLE & AUNTY FAMILY RESTAURANT',
 'Unatoto Guoco Tower',
 'VENUE By Sebastian',
 'The Social Kitchen',
 "Oscar's",
 'The Orchard Cafe',
 'Sampoorna Swadesi Vegetarian Restaurant',
 'WINESTONE',
 'WAKUDA',
 'White Rose Cafe',
 'Willow',
 'Yang Guo Fu Mala Tang 杨国福麻辣烫 @ Northpoint City',
 'YinJi Singapore',
 'YUAN 原',
 'Ru Yi Yuan Vegetarian',
 'Ting Yuan Hotpot Buffet',
 'Yue Bai',
 'Ya Ge - 雅阁 (Orchid Hotel SG)',
 'Yì By Jereme Leung',
 'Zén',
 'nakey',
 'Sibarita bar & resto',
 'Sibarita Restaurante',
 'Sibarita',
 'Sibarita Habana',
 'tiram',
 'The Fullerton Pavillion',
 'True Cost',
 '极鲜 Ji Xian Seafood & Hong Kong Dim Sum Express',
 '老四川豆花庄',
 '麻辣诱惑 Spice Spirits Singapore',
 'Huda Restaurant SG 胡大饭店',
 'Huevos @ River Valley',
 'Humpback',
 'ITOKO JAPANESE RESTAURANT',
 'IVINS @ Thomson Plaza',
 'Adipoli Curry Pot',
 'Indian Tandoor Restaurant',
 'Indo Rasa Singapore',
 'Indobowl, Singapore @ Sultan Gate',
 'Waroeng Anak Indo',
 "D'Penyetz Jem",
 'Samrang SG',
 'Ingleside',
 'Isshin Machi Greenwood',
 'JAAN By Kirk Westaway',
 'Jang San Korean Restaurant 장산왕족발',
 'Jia Xiang Sarawak Kuching Kolo Mee',
 'Jia Xiang Sarawak Kuching Kolo Mee - Marina Square',
 'Jia Xiang Sarawak Kuching Kolo Mee - Alexandra',
 'Jiak Modern Tzechar @ Hillv2',
 'Joséphine - French Cuisine & Wine Bar',
 'Kap Kun Kap Thai & Seafood',
 'Kazan Japanese Cuisine',
 'Kim Poh FishHead Steamboat',
 'fennel by komala vilas',
 'Korat Thai Cafe',
 "LET'S KINN THAI",
 "la table d'Emma",
 'Lao You Ji Fishhead Steamboat Seafood Restaurant',
 'Lazy Lizard - Flora Vista',
 'Le Petit Chef - Espoir Singapore',
 'Lepak at Sultan',
 'Lolla',
 "Lynnette's Kitchen (Private Dining)",
 'Lẩu Dê 3 Cô',
 '这边来 Zhe bian lai',
 'RED DOT MALA HOTPOT 小红点麻辣香锅',
 'Ri Ri Sheng Mala Hotpot',
 'Xing Le Mala Hotpot',
 'Madame Fan',
 'Makan Japanese restaurant',
 'MCC MAKAN PLACE',
 'Makan@Jen',
 'Miyu',
 'TAP @ 9 Penang',
 'Wild Coco (Hamilton Road)',
 'Seroja',
 'Mamacitas',
 'Man Man Japanese Unagi (Clarke Quay)',
 'Man Man Japanese Unagi (Duo Galleria)',
 'Marguerite',
 'Meta Restaurant',
 'True Veggie',
 'NAEUM Restaurant',
 'Naga House',
 'Nan Hua Chang (Traditional) Fish Head Steamboat',
 'Nicolas Le Restaurant',
 'Nyonyas & Gentlemen Restaurant',
 'Ocean Curry Fish Head',
 'ODETTE',
 'Restaurant Odette',
 'Odette',
 'Buffet Odette',
 "Old Bibik's Peranakan Kitchen",
 'Olivia Restaurant & Lounge',
 'Paofan Paradise 泡饭天 - T.K Kitchen Restaurant',
 'Papa Ayam 313@somerset',
 'Parkway Mini Steamboat',
 'Park Bäckerei',
 "Lim's Penang Prawn Noodle and Fried Kway Teow",
 'Milligram',
 'Podi & Poriyal',
 'Putra Minang Restaurant Bencoolen',
 'Ăn Là Ghiền -Buffet 5',
 'Qi - House of Sichuan',
 'Qi Xiang Hotpot',
 'Rainbow Rice House',
 '123 ZÔ - Ẩm Thực Việt',
 '27 Mezze Bar and Grill',
 '57° Mala Xiang Guo @Ang Mo Kio',
 '57° Mala Xiang Guo @Shaw Plaza',
 '87 Club Street',
 '88 Hong Kong Roast Meat Specialist',
 '88 Hong Kong Roast Meat Specialist',
 'ASIAN MARKET CAFE',
 'Absurdities Singapore',
 'Abundance: Taiwanese Restaurant & Craft Beer Bar (Redhill)',
 "Ajumma's Northpoint",
 "Ajumma's Westgate",
 'Alegria Singapore',
 'Alice Boulangerie & Restaurant',
 'Restoran Al Alif, Bandar Seri Alam',
 'Anglo Indian Cafe & Bar',
 'Anglo Indian Cafe & Bar',
 'Apéro',
 'Arab Street Turkish & Western Restaurant',
 'Hup Lee Cafeteria (Arcade Fish Soup)',
 'Ayam Penyet President Express - Midtown Hougang',
 'BACARO Italian Restaurant and Wine Bar',
 'Bacos',
 'Badaro Korean Seafood',
 'Bangkok Thai Muslim',
 'Bearded Bella',
 'Beast Duxton',
 'Bee Keong Restaurant',
 'Bella pizzeria',
 'Berlin65',
 'Bugis BBQ and Hotpot',
 'Bugis Fish Head Steamboat',
 'Buko Nero',
 'Thien Long Vietnamese Food',
 'Star Pho Le Beef Noodle Soup ( Subsidiary of Banh Mi Thit by Star Baguette )',
 "Làng Nướng 1980's - Authentic Vietnamese home-cooked Cuisine",
 'Ăn Là Ghiền-Cơm Niêu',
 'COL',
 'COLLECTIVE',
 'Well Collective',
 'Cafe de Muse - The Restaurant',
 'Caffe Fernet',
 'Cendra Tapas Bar and Grill',
 "Charlie's Restaurant & Bar (Boat Quay)",
 "Chef's Table by Chef Stephan (Western Omakase, Fine Dining)",
 "Chen's",
 'Chengdu Bowl Tanjong Pagar 成都Bowl',
 'Cherry & Oak',
 'Wannian Stonepot Fish',
 'Yuet Sing Seafood',
 'Yi Zun Noodle @ Marina Square',
 'Alijiang',
 'Penang Culture',
 'Western Mahua',
 'GLC Hao Yun Lai Restaurant (好运来餐厅）',
 'Al Madinah Restaurant',
 'Overrice (Voted Best Halal Restaurant near Haji Lane)',
 'Selera Serambi by Tang Tea House',
 'Bumbu Restaurant',
 'Chuan Kee Seafood',
 'Chuan Kee Seafood Toa Payoh',
 'Colonial Club Signatures',
 'Comida Mexicana Singapore',
 'Crafted by Peter Zwiener',
 'DaXi 大喜 - VivoCity',
 'Dapur Penyet @ City Plaza',
 'JB Garden Seafood Restaurant',
 'Din Tai Fung',
 'Dum Pukht Grill & Curry',
 'Dungeon by IL Fiore',
 'Ristorante Vista Duomo',
 'El Mesa Filipino Seafood & Grill Clarke Quay',
 'Element Restaurant Amara Singapore',
 'Ellenborough Market Café',
 'Empire Hotpot & Cantonese Cuisine',
 'Estuary Seafood Restaurant Bar - Event Space Orchard Road',
 'Cherki',
 'Olive Vine Pasta Fusion',
 'Tambi Restaurant',
 "Martin's Kitchen",
 'Quay House',
 'ilmiri Korean Fusion Cuisine',
 'LECOQ',
 '15 Stamford',
 'BYD by DXD',
 'Kubo Wood-fired Cuisine & Wine Bar',
 'Synthesis 食拿酒稳: Modern Chinese Restaurant & Cocktail Bar (Suntec City)',
 'RACINES',
 'Tash Tish Tosh @ 50K',
 'JU95',
 'Fairytail Bar - By The Halal Mixologist',
 'Feng Sheng Kampong Chicken Rice & Steamboat (24 Hours) | 豐盛滑鸡火锅',
 'Fish & Bones',
 '#Foodcoholic',
 'Frieda Beer Garden & German Restaurant',
 'Frog Meat Fish Head 美蛙魚頭自助火锅 和炒菜',
 'Godmama at Parkway Parade - Modern Peranakan Restaurant Singapore',
 'Gold Ocean Curry Fish Head',
 'Gong Yuan Ma La Tang',
 'Gong Yuan Ma La Tang 宫源麻辣烫 @ 18 Tai Seng',
 'Good Luck BBQ 上上签四川小酒馆',
 'Great Nanyang Heritage Cafe 大南洋',
 'Guccio Ristorante @Telok Ayer',
 'Hahahotpot Jalan Pisang',
 'Abang Dol',
 'Restoran Ke Ren Lai',
 '109 Yong Tau Foo',
 'Jason Niang Dou Fu',
 'Orchard Yong Tau Fu',
 'The Beef House',
 'Gem Fresh Yong Tau Foo',
 'Bagus LaMian',
 'Thunderbolt Tea by Boon Lay Traditional Hakka Lui Cha',
 'Hakka Yong Tau Foo',
 'Hannibal Singapore - European Grill and Bar',
 'HarriAnns Nonya Table',
 'Genius Central Singapore',
 'Hei Lun Shi Tang 黑轮食堂',
 'Hong Fan Tian 红翻天 Mala Hotpot',
 'Hopscotch (Capitol)']

In [4]:
len(restaurant_cats)

295

In [5]:
len(RESTAURANT_RECATEGORIZATION)

299

In [3]:
data = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/all_restaurants_dataset.csv')

In [9]:
data = data.drop(columns=['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'])
data = data.drop_duplicates(subset=['place_id', 'name', 'address'], keep='last')
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6580 entries, 0 to 10103
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       6580 non-null   object 
 1   name           6580 non-null   object 
 2   reviews        6580 non-null   int64  
 3   main_category  6580 non-null   object 
 4   categories     6580 non-null   object 
 5   main_rating    6580 non-null   float64
 6   address        6580 non-null   object 
 7   link           6580 non-null   object 
 8   review_photos  6580 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 514.1+ KB


In [10]:
data[data.place_id =='ChIJUaQyC0QZ2jERfBgqiBnCpls']

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
9261,ChIJUaQyC0QZ2jERfBgqiBnCpls,Thunderbird Bistro,192,Asian fusion restaurant,"['Asian fusion restaurant', 'Beer garden', 'Co...",4.4,"11 Unity Street, Robertson Walk, #01-10 11, 23...",https://www.google.com/maps/place/Thunderbird+...,['https://lh5.googleusercontent.com/p/AF1QipMx...


In [20]:
data = data[~data['name'].isin(NAMES_TO_DROP)]
data = data[~data['place_id'].isin(PLACE_ID_TO_DROP)]

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6561 entries, 0 to 10103
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       6561 non-null   object 
 1   name           6561 non-null   object 
 2   reviews        6561 non-null   int64  
 3   main_category  6561 non-null   object 
 4   categories     6561 non-null   object 
 5   main_rating    6561 non-null   float64
 6   address        6561 non-null   object 
 7   link           6561 non-null   object 
 8   review_photos  6561 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 512.6+ KB


In [30]:
data = data[(data['main_rating'] >= 3.7) & (data['reviews'] >=75) & (data['reviews'] <= 2000)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3380 entries, 0 to 10099
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       3380 non-null   object 
 1   name           3380 non-null   object 
 2   reviews        3380 non-null   int64  
 3   main_category  3380 non-null   object 
 4   categories     3380 non-null   object 
 5   main_rating    3380 non-null   float64
 6   address        3380 non-null   object 
 7   link           3380 non-null   object 
 8   review_photos  3380 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 264.1+ KB


In [31]:
data['categories'] = data['categories'].apply(lambda x: ast.literal_eval(x))
sec_cat_mask = (data['main_category'] == 'Restaurant') & (data['categories'].apply(lambda x: x != ['Restaurant']))
data.loc[sec_cat_mask, 'main_category'] = data['categories'].apply(lambda x: x[1] if len(x) > 1 else x[0])

In [32]:
data[data.main_category=='Restaurant']

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
26,ChIJueAjEfEZ2jER63xqRZHiAKE,Restaurant Poise,114,Restaurant,[Restaurant],4.9,"6 Teck Lim Rd, Singapore 088384",https://www.google.com/maps/place/Restaurant+P...,['https://lh5.googleusercontent.com/p/AF1QipNg...
29,ChIJDxn4XBYZ2jERrbHE7aDSKGY,Revolver,298,Restaurant,[Restaurant],4.3,"56 Tras St, Singapore 078995",https://www.google.com/maps/place/Revolver/dat...,['https://lh5.googleusercontent.com/p/AF1QipMq...
68,ChIJdy8H8KMZ2jERhxJ2kuj3NH8,Rod Dee Thai Cafe,143,Restaurant,[Restaurant],3.9,"5 Teo Hong Rd, MRT Exit 4, Singapore 088323",https://www.google.com/maps/place/Rod+Dee+Thai...,['https://lh5.googleusercontent.com/p/AF1QipPr...
124,ChIJ4fQILgAZ2jERsJ8s130tNv0,Dickson Nasi Lemak,78,Restaurant,[Restaurant],4.1,"12 Gopeng St, #01-89/90, icon 078877",https://www.google.com/maps/place/Dickson+Nasi...,['https://lh5.googleusercontent.com/p/AF1QipPS...
128,ChIJNVinX68Z2jERWFXDwnDd9ZA,SKAI Restaurant,793,Restaurant,[Restaurant],4.1,"2 Stamford Rd, Lvl 70 #70-01, Singapore 178882",https://www.google.com/maps/place/SKAI+Restaur...,['https://lh5.googleusercontent.com/p/AF1QipMn...
...,...,...,...,...,...,...,...,...,...
9873,ChIJWco1mvwZ2jERixaRGmkRB9w,HarriAnns Nonya Table,336,Restaurant,[Restaurant],3.9,"3 Temasek Blvd, #01-416A, Suntec City West Win...",https://www.google.com/maps/place/HarriAnns+No...,['https://lh5.googleusercontent.com/p/AF1QipPX...
9934,ChIJ6SJM5hQZ2jERGAGlT3FpUNw,Genius Central Singapore,590,Restaurant,[Restaurant],4.0,"01-01 7-13 Amoy Street Far east square, 049949",https://www.google.com/maps/place/Genius+Centr...,['https://lh5.googleusercontent.com/p/AF1QipOF...
9994,ChIJoeBXbssb2jERsnEOvlPtBrQ,Hei Lun Shi Tang 黑轮食堂,101,Restaurant,[Restaurant],4.1,"1 Queensway, #02-47, Singapore 149053",https://www.google.com/maps/place/Hei+Lun+Shi+...,['https://lh5.googleusercontent.com/p/AF1QipPv...
10047,ChIJXRU4LoYX2jERxWteoisrRZU,Hong Fan Tian 红翻天 Mala Hotpot,92,Restaurant,[Restaurant],3.8,"blk 190 Lor 6 Toa Payoh, #01-528, Singapore 31...",https://www.google.com/maps/place/Hong+Fan+Tia...,['https://lh5.googleusercontent.com/p/AF1QipNZ...


In [13]:
list(data[data.main_category=='Restaurant'].name)

['Restaurant Poise',
 'Revolver',
 'Rod Dee Thai Cafe',
 'Dickson Nasi Lemak',
 'SKAI Restaurant',
 'SUSHISAMBA Singapore',
 'Saap Saap Thai',
 'Salt & Pepper Pte Ltd',
 'San Laksa Steamboat Pte Ltd',
 'San Shu Gong Private Dining 三叔公私房菜',
 'Sanook Kitchen - West Gate',
 'Sanook Kitchen - City Square',
 'Sakari Sushi',
 'Warung Selera Masakan Kampung',
 'Selera Utama Kampung Plentong Baru (Sup Power)',
 'Selera Kampung',
 'Restoran Selera Sambal',
 'Medan Selera Kampung Tengah Gedok',
 'Restoran Selera Kampung',
 'Restoran Selera Kampung (FAM MAJU)',
 'SHUNFENG 顺风川渝私房菜',
 '蜀都丰 SO DO FUN @ MIDTOWN HOUSE',
 'Hibiscus Tree (芙蓉树下) Millenia Walk',
 '天府李川菜 DAINTY SICHUAN',
 '辣不辣 Labula BBQ Sichuan Restaurant @ Hougang Village',
 'Fukurou Kitchen',
 'Sin Manbok Korean Restaurant 신만복',
 'Sinpopo Brand Restaurant @ TANGS',
 'Siri House Dempsey',
 'Sky View Pavilion',
 'Soi 19 Thai Wanton Mee Telok Ayer',
 'Soon Huat (JB) Bak Kut Teh',
 'Steamboat Charcoal Claypot Prawn @Toa Payoh',
 'Crab Wang 

In [33]:
restaurant_cat_mask = data['main_category'] == 'Restaurant'
data.loc[restaurant_cat_mask, 'main_category'] = data['name'].apply(lambda x: RESTAURANT_RECATEGORIZATION.get(x, 'Restaurant'))

In [34]:
cats_to_remove = [cat.lower() for cat in CATS_TO_REMOVE]
mask = data['main_category'].str.lower().isin(cats_to_remove)
data = data[~mask]

In [35]:
data['main_category'] = data['main_category'].apply(lambda x: x.replace(' restaurant', ''))

In [36]:
data[data.main_category=='Restaurant']

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos


In [37]:
data['cuisine'] = data['main_category'].apply(lambda x: CATEGORY_TO_CUISINE[x])

In [38]:
data[data.place_id =='ChIJUaQyC0QZ2jERfBgqiBnCpls']

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,cuisine
9261,ChIJUaQyC0QZ2jERfBgqiBnCpls,Thunderbird Bistro,192,Asian fusion,"[Asian fusion restaurant, Beer garden, Cocktai...",4.4,"11 Unity Street, Robertson Walk, #01-10 11, 23...",https://www.google.com/maps/place/Thunderbird+...,['https://lh5.googleusercontent.com/p/AF1QipMx...,Fusion


In [19]:
data.main_category.unique()

array(['Seafood', 'Singaporean', 'Fine dining', 'European', 'Thai',
       'French', 'Chinese', 'Indian', 'Italian', 'Mediterranean', 'Pizza',
       'Family', 'Korean', 'Grill', 'Halal', 'Vegetarian', 'Steak house',
       'Japanese', 'Hamburger', 'Malaysian', 'Bar', 'Western',
       'Traditional', 'Fusion', 'Tapas', 'Chinese noodle', 'Pakistani',
       'Cafe', 'Modern French', 'Tex-Mex', 'Local', 'Mexican', 'Andhra',
       'Seafood donburi', 'Fish and seafood', 'Sushi', 'Modern izakaya',
       'Conveyor belt sushi', 'Izakaya', 'Oyster bar', 'Bistro', 'Buffet',
       'Bakso', 'Authentic Japanese', 'Indonesian',
       'Cha chaan teng (Hong Kong-style cafe)', 'Tempura donburi',
       'Korean barbecue', 'Taco', 'Sukiyaki and Shabu Shabu',
       'Shabu-shabu', 'Hot pot', 'Russian', 'Taiwanese', 'Modern Indian',
       'Teppanyaki', 'Yakitori', 'Barbecue', 'Sichuan', 'Asian',
       'Creperie', 'American', 'Vietnamese', 'Cantonese', 'Soup shop',
       'Soup', 'Asian fusion', 'Art 

In [20]:
data.cuisine.nunique()

36

In [21]:
set(data.main_category)

{'African',
 'American',
 'Andhra',
 'Argentinian',
 'Art cafe',
 'Asian',
 'Asian fusion',
 'Australian',
 'Authentic Japanese',
 'Bakso',
 'Bar',
 'Bar & grill',
 'Barbecue',
 'Basque',
 'Biryani',
 'Bistro',
 'Brasserie',
 'Brazilian',
 'Breakfast',
 'Brunch',
 'Buffet',
 'Cafe',
 'Café',
 'Cantonese',
 'Cat cafe',
 'Catalonian',
 'Cha chaan teng (Hong Kong-style cafe)',
 'Chettinad',
 'Chicken',
 'Chinese',
 'Chinese noodle',
 'Cocktail bar',
 'Conveyor belt sushi',
 'Crab house',
 'Creperie',
 'Delivery Chinese',
 'Dessert',
 'Dim sum',
 'Diner',
 'Dumpling',
 'Eastern European',
 'European',
 'Family',
 'Filipino',
 'Fine dining',
 'Fish and seafood',
 'French',
 'French steakhouse',
 'Fried chicken takeaway',
 'Fusion',
 'Gastropub',
 'German',
 'Gluten-free',
 'Greek',
 'Grill',
 'Hakka',
 'Halal',
 'Hamburger',
 'Health food',
 'Hong Kong style fast food',
 'Hot pot',
 'Hunan',
 'Indian',
 'Indian Muslim Restaurant',
 'Indonesian',
 'International',
 'Israeli',
 'Italian',
 'I

In [22]:
###############

In [2]:
data = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/all_restaurants_dataset.csv')


In [3]:
data = clean_restaurant_data(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          2706 non-null   object 
 1   name              2706 non-null   object 
 2   reviews           2706 non-null   int64  
 3   main_category     2706 non-null   object 
 4   main_rating       2706 non-null   float64
 5   address           2706 non-null   object 
 6   link              2706 non-null   object 
 7   review_photos     2706 non-null   object 
 8   cuisine           2706 non-null   object 
 9   latitude          2706 non-null   float64
 10  longtitude        2706 non-null   float64
 11  full_postal_code  2706 non-null   object 
 12  postal_code       2706 non-null   object 
 13  district_code     2706 non-null   object 
 14  region            2706 non-null   object 
dtypes: float64(3), int64(1), object(11)
memory usage: 317.2+ KB


In [5]:
data.main_category.nunique()

149

In [5]:
reviews_v1 = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/reviews_data/all_reviews_dataset_v1.csv', low_memory=False)

In [3]:
reviews_v1

,place_id,name,review_id,rating,review_text,published_at,published_at_date,review_likes_count,response_from_owner_text,response_from_owner_ago,response_from_owner_date,total_number_of_reviews_by_reviewer,total_number_of_photos_by_reviewer,is_local_guide,review_translated_text,response_from_owner_translated_text
0,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSUM3bE9DZ2lRRRAB,5,Delicious delicious delicious! Had the Tom yum...,2 weeks ago,2024-08-15 00:17:31.766742,0,NaN,NaN,NaN,453.0,1210.0,True,NaN,NaN
1,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChZDSUhNMG9nS0VJQ0FnSURicHQyb2Z3EAE,4,Fairly authentic Pad Thai cooked by a Thai per...,3 weeks ago,2024-08-08 00:17:31.769063,0,NaN,NaN,NaN,1381.0,8425.0,True,NaN,NaN
2,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSURieXYtTGdnRRAB,5,I had the fried kway tiao which cost $6. If ha...,3 weeks ago,2024-08-08 00:17:31.771272,0,NaN,NaN,NaN,615.0,2361.0,True,NaN,NaN
3,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSUNiOFlhSzl3RRAB,3,I got the prawn omelette rice here for $6. The...,a month ago,2024-07-29 00:17:31.773506,0,NaN,NaN,NaN,29.0,40.0,False,NaN,NaN
4,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChZDSUhNMG9nS0VJQ0FnSUN6NnJhZEp3EAE,5,Really tasty Thai place in chinatown! The gree...,2 months ago,2024-06-29 00:17:31.775457,0,NaN,NaN,NaN,38.0,12.0,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148069,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSUNRZ0plZ2l3RRAB,4,Vegetarian eatery with good range of dishes th...,8 years ago,2016-08-30 15:41:51.335461,0,NaN,NaN,NaN,100.0,189.0,True,NaN,NaN
4148070,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSURBc2VmdGt3RRAB,4,Very nice vegetarian porridge and side dishes,8 years ago,2016-08-30 15:41:51.339127,0,NaN,NaN,NaN,344.0,1137.0,True,NaN,NaN
4148071,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSUNRLXN5MG1BRRAB,1,expensive and not nice!,8 years ago,2016-08-30 15:41:51.342079,2,NaN,NaN,NaN,14.0,NaN,False,NaN,NaN
4148072,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChZDSUhNMG9nS0VJQ0FnSURnM3FMWWN3EAE,5,Cheap and delicious,8 years ago,2016-08-30 15:41:51.345018,0,NaN,NaN,NaN,29.0,48.0,True,NaN,NaN


In [4]:
reviews_df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/reviews_data/all_reviews_dataset.csv', low_memory=False)

In [4]:
reviews = reviews_v1.copy()
reviews

,place_id,name,review_id,rating,review_text,published_at,published_at_date,review_likes_count,response_from_owner_text,response_from_owner_ago,response_from_owner_date,total_number_of_reviews_by_reviewer,total_number_of_photos_by_reviewer,is_local_guide,review_translated_text,response_from_owner_translated_text
0,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSUM3bE9DZ2lRRRAB,5,Delicious delicious delicious! Had the Tom yum...,2 weeks ago,2024-08-15 00:17:31.766742,0,NaN,NaN,NaN,453.0,1210.0,True,NaN,NaN
1,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChZDSUhNMG9nS0VJQ0FnSURicHQyb2Z3EAE,4,Fairly authentic Pad Thai cooked by a Thai per...,3 weeks ago,2024-08-08 00:17:31.769063,0,NaN,NaN,NaN,1381.0,8425.0,True,NaN,NaN
2,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSURieXYtTGdnRRAB,5,I had the fried kway tiao which cost $6. If ha...,3 weeks ago,2024-08-08 00:17:31.771272,0,NaN,NaN,NaN,615.0,2361.0,True,NaN,NaN
3,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSUNiOFlhSzl3RRAB,3,I got the prawn omelette rice here for $6. The...,a month ago,2024-07-29 00:17:31.773506,0,NaN,NaN,NaN,29.0,40.0,False,NaN,NaN
4,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChZDSUhNMG9nS0VJQ0FnSUN6NnJhZEp3EAE,5,Really tasty Thai place in chinatown! The gree...,2 months ago,2024-06-29 00:17:31.775457,0,NaN,NaN,NaN,38.0,12.0,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148069,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSUNRZ0plZ2l3RRAB,4,Vegetarian eatery with good range of dishes th...,8 years ago,2016-08-30 15:41:51.335461,0,NaN,NaN,NaN,100.0,189.0,True,NaN,NaN
4148070,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSURBc2VmdGt3RRAB,4,Very nice vegetarian porridge and side dishes,8 years ago,2016-08-30 15:41:51.339127,0,NaN,NaN,NaN,344.0,1137.0,True,NaN,NaN
4148071,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSUNRLXN5MG1BRRAB,1,expensive and not nice!,8 years ago,2016-08-30 15:41:51.342079,2,NaN,NaN,NaN,14.0,NaN,False,NaN,NaN
4148072,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChZDSUhNMG9nS0VJQ0FnSURnM3FMWWN3EAE,5,Cheap and delicious,8 years ago,2016-08-30 15:41:51.345018,0,NaN,NaN,NaN,29.0,48.0,True,NaN,NaN


In [5]:
reviews_df = clean_reviews_df(reviews)

reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1348619 entries, 0 to 4147825
Data columns (total 11 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   place_id                             1348619 non-null  object 
 1   name                                 1348619 non-null  object 
 2   review_id                            1348619 non-null  object 
 3   rating                               1348619 non-null  int64  
 4   review_text                          1348619 non-null  object 
 5   published_at_date                    1348619 non-null  object 
 6   review_likes_count                   1348619 non-null  int64  
 7   response_from_owner_text             1348619 non-null  object 
 8   total_number_of_reviews_by_reviewer  1348619 non-null  float64
 9   total_number_of_photos_by_reviewer   1348619 non-null  float64
 10  is_local_guide                       1348619 non-null  bool   
dty

In [6]:
reviews.columns

Index(['place_id', 'name', 'review_id', 'rating', 'review_text',
       'published_at', 'published_at_date', 'review_likes_count',
       'response_from_owner_text', 'response_from_owner_ago',
       'response_from_owner_date', 'total_number_of_reviews_by_reviewer',
       'total_number_of_photos_by_reviewer', 'is_local_guide',
       'review_translated_text', 'response_from_owner_translated_text'],
      dtype='object')

In [28]:
reviews_df = reviews.drop_duplicates()
reviews_df = reviews_df.drop(columns=['review_translated_text', 'response_from_owner_translated_text', 'response_from_owner_ago', 'response_from_owner_date', 'published_at'])

In [ ]:
data.to_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/cleaned_all_restaurants_dataset.csv', index=False, header=True)

In [ ]:
data = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/cleaned_all_restaurants_dataset.csv', dtype={'full_postal_code': str, 'postal_code':str, 'district_code':str})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4603 entries, 0 to 4602
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          4603 non-null   object 
 1   name              4603 non-null   object 
 2   reviews           4603 non-null   int64  
 3   main_category     4603 non-null   object 
 4   categories        4603 non-null   object 
 5   main_rating       4603 non-null   float64
 6   address           4603 non-null   object 
 7   link              4603 non-null   object 
 8   review_photos     4603 non-null   object 
 9   coordinates       4603 non-null   object 
 10  latitude          4603 non-null   float64
 11  longtitude        4603 non-null   float64
 12  full_postal_code  4603 non-null   object 
 13  postal_code       4603 non-null   object 
 14  district_code     4602 non-null   object 
 15  region            4602 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory

In [ ]:
#drop duplicates with subset 'place_id', 'name', 'reviews', 'address']
df = df.drop_duplicates(subset=['place_id', 'name', 'reviews', 'address'])

# remove irrelevant place based on main_categories
cats_to_remove = [cat.lower() for cat in CATS_TO_REMOVE]
mask = df['main_category'].str.lower().isin(cats_to_remove)
df = df[~mask]

# remove irrelevant place based by name
#df = df[~df['name'].isin(NAME_TO_DROP)]
df = df[~df['place_id'].isin(PLACE_ID_TO_DROP)]

# fill na in main_category and categories(as ['unknown'])
df['main_category'].fillna('unknown', inplace=True)
df['main_category'] = df['main_category'].apply(lambda x: x.replace(' restaurant', ''))
df['categories'] = df['categories'].fillna("['unknown']").apply(ast.literal_eval)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4797 entries, 0 to 10093
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       4797 non-null   object 
 1   name           4797 non-null   object 
 2   reviews        4797 non-null   int64  
 3   main_category  4797 non-null   object 
 4   categories     4797 non-null   object 
 5   main_rating    4797 non-null   float64
 6   address        4771 non-null   object 
 7   link           4797 non-null   object 
 8   review_photos  4797 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 374.8+ KB


In [ ]:
df.to_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/final_restaurant_b1.csv', index=False, header=True)

In [ ]:
contains_pte_ltd = df['name'].str.contains('Pte Ltd', case=False, na=False)
contains_pte_ltd.sum()

52

In [ ]:
unknown_main_cat = df['main_category'] == 'unknown'
unknown_main_cat.sum()

40

In [ ]:
no_rating = df['main_rating'] == 0
no_rating.sum()

215

In [ ]:
restaurant = df['main_category'].str.contains('restaurant', case=False, na=False)
restaurant.sum()

4150

In [ ]:
to_remove = df[(~restaurant & contains_pte_ltd & no_rating) | unknown_main_cat].reset_index(drop=True)
to_remove.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       40 non-null     object 
 1   name           40 non-null     object 
 2   reviews        40 non-null     int64  
 3   main_category  40 non-null     object 
 4   categories     40 non-null     object 
 5   main_rating    40 non-null     float64
 6   address        38 non-null     object 
 7   link           40 non-null     object 
 8   review_photos  40 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 2.9+ KB


In [ ]:
to_remove

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
0,ChIJg0UVB4IZ2jERfWKNEG8EyDc,Smile Dessert,0,unknown,[unknown],0.0,"717 Havelock Rd, Singapore 160022",https://www.google.com/maps/place/Smile+Desser...,[]
1,ChIJScNPoUKjAmAR3OO48IT78tQ,Takayama,0,unknown,[unknown],0.0,"Gifu, Japan","https://www.google.com/maps/place/Takayama,+Gi...",[]
2,ChIJc-gZD5EZ2jERRuJ5h8wWLH4,Thaiexpress Concepts Cineleisure Orchard,1,unknown,[unknown],3.0,"8 Grange Rd, Cineleisure Orchard, Singapore 23...",https://www.google.com/maps/place/Thaiexpress+...,[]
3,ChIJibiOH5kZ2jERcW3MZYgn_RI,The Sushi Bar Dining,1,unknown,[unknown],1.0,"280 River Valley Rd, Singapore 238321",https://www.google.com/maps/place/The+Sushi+Ba...,[]
4,ChIJyWPGSAIZ2jERw7arCVkYYdY,Vietnamese Cuisine,3,unknown,[unknown],4.7,"7 Hoe Chiang Rd, Singapore 089313",https://www.google.com/maps/place/Vietnamese+C...,['https://lh5.googleusercontent.com/p/AF1QipOe...
5,ChIJNcn6xtYb2jERKTlNIZRmQis,Ma La Xiang Guo,0,unknown,[unknown],0.0,"164 Bukit Merah Central, Singapore 150164",https://www.google.com/maps/place/Ma+La+Xiang+...,[]
6,ChIJPTjULjF0K4cRDfdv8pqL2ec,Rise,0,unknown,[unknown],0.0,"Scottsdale, AZ 85260, United States",https://www.google.com/maps/place/Rise/data=!4...,[]
7,ChIJrwRa3javK4cRKsgBw7yd65c,Rise,0,unknown,[unknown],0.0,"Mesa, AZ 85206, United States",https://www.google.com/maps/place/Rise/data=!4...,[]
8,ChIJ1239LW0Z2jERjAM826R9LvI,Amò,1,unknown,[unknown],4.0,"57 Tras St, Singapore 078996",https://www.google.com/maps/place/Am%C3%B2/dat...,[]
9,ChIJa1-WpnIZ2jERHQAr226M9eo,Amò,0,unknown,[unknown],0.0,"18 Tg Pagar Rd, Singapore 088441",https://www.google.com/maps/place/Am%C3%B2/dat...,[]


In [ ]:
df['main_category'] = df['main_category'].apply(lambda x: x.replace(' restaurant', ''))

In [ ]:
df.main_category.unique()

array(['Seafood', 'Vietnamese', 'Cantonese', 'Indian', 'Restaurant',
       'Dim sum', 'Cafe', 'Singaporean', 'Indonesian', 'Halal',
       'Fine dining', 'Modern European', 'Asian fusion', 'Thai', 'Nyonya',
       'Chinese', 'French', 'Hot pot', 'Italian', 'Vegetarian',
       'Mediterranean', 'Meat dish', 'Pizza', 'Family', 'Korean',
       'Japanese', 'Grill', 'Steak house', 'Sushi', 'Hamburger', 'Asian',
       'Bar', 'Traditional', 'Tapas', 'Mexican', 'Chinese noodle',
       'Pakistani', 'Sichuan', 'Turkish', 'Modern French', 'Salad shop',
       'Seafood donburi', 'Tex-Mex', 'Chicken', 'South Indian', 'Andhra',
       'Lebanese', 'Fish and seafood', 'Izakaya', 'Authentic Japanese',
       'Modern izakaya', 'Conveyor belt sushi', 'Unagi', 'Oyster bar',
       'Bistro', 'Steamboat', 'Indian Muslim Restaurant', 'Malaysian',
       'Nasi', 'Brunch', 'Bakso', 'Breakfast', 'Fried chicken takeaway',
       'Padang', 'Ramen', 'Cha chaan teng (Hong Kong-style cafe)',
       'Tempura donb

In [ ]:
df

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
0,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,"[Seafood restaurant, Asian restaurant, Dim sum...",4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...
1,ChIJP2EDPqAZ2jERJKipSQq7qY8,Red House Seafood Nanyang (Clarke Quay),730,Seafood,[Seafood restaurant],4.1,"3C River Valley Road 01-02/03, The Cannery, 17...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipMe...
2,ChIJSW7NnhAZ2jERj8p3W_nnhGE,Red House Seafood at Esplanade,186,Seafood,"[Seafood restaurant, Chinese restaurant]",4.4,"8 Raffles Ave., #01-14 / 16 Esplanade, Singapo...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipOJ...
3,ChIJNxLu-EAb2jER_a4Y4ahn8TU,Red Sparrow,496,Vietnamese,"[Vietnamese restaurant, Asian restaurant]",4.0,"11 Dempsey Rd, #01-18, 249673, Singapore 249673",https://www.google.com/maps/place/Red+Sparrow/...,['https://lh5.googleusercontent.com/p/AF1QipO8...
4,ChIJIds6rnUZ2jERPx6E0UCf6HM,Red Star Restaurant,3725,Cantonese,"[Cantonese restaurant, Chinese restaurant, Dim...",3.7,"Block 54 Chin Swee Rd, #07-23, Singapore 160054",https://www.google.com/maps/place/Red+Star+Res...,['https://lh5.googleusercontent.com/p/AF1QipN-...
...,...,...,...,...,...,...,...,...,...
10089,ChIJA2UPtIwZ2jERE6WwKP0x278,Hua Ting Steamboat,70,Steamboat,"[Steamboat restaurant, Chinese restaurant]",4.2,"Mezzanine level #01-08, Claymore Connect, 442 ...",https://www.google.com/maps/place/Hua+Ting+Ste...,['https://lh5.googleusercontent.com/p/AF1QipM4...
10090,ChIJ3XkDF-EW2jERnoxsx_i5CcY,Hualong Fishhead Steamboat,173,Steamboat,"[Steamboat restaurant, Asian restaurant, Chine...",3.8,"347 Ang Mo Kio Ave 3, #01-2142 Blk 347, Singap...",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMY...
10091,ChIJLfNb-H0X2jERHknheYoXfgY,Hualong Fishhead Steamboat,45,Chinese,[Chinese restaurant],3.8,"978 Toa Payoh N, #02-01, Singapore 319001",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMl...
10092,ChIJ6-YzAysX2jERKmeXREbO7VM,Hualong Fishhead Steamboat,8,Chinese,[Chinese restaurant],2.6,"Blk 631 Hougang Ave 8, #01-06, Singapore 530631",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipOW...


In [ ]:
df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/all_restaurants_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10104 entries, 0 to 10103
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       10104 non-null  object 
 1   name           10104 non-null  object 
 2   reviews        10104 non-null  int64  
 3   main_category  10015 non-null  object 
 4   categories     10015 non-null  object 
 5   main_rating    10104 non-null  float64
 6   address        9992 non-null   object 
 7   link           10104 non-null  object 
 8   review_photos  10104 non-null  object 
 9   Unnamed: 9     0 non-null      float64
 10  Unnamed: 10    0 non-null      float64
 11  Unnamed: 11    0 non-null      float64
 12  Unnamed: 12    0 non-null      float64
 13  Unnamed: 13    0 non-null      float64
 14  Unnamed: 14    0 non-null      float64
 15  Unnamed: 15    0 non-null      float64
 16  Unnamed: 16    0 non-null      float64
dtypes: float64(9), int64(1), object(7)
memory usage: 1

In [ ]:
#drop irrelevant columns, rows with null values, df duplicates with subset 'place_id', 'name', 'reviews', 'address']
df = df.drop(columns=['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'])
df = df.dropna()

# remove irrelevant place based on main_categories
cats_to_remove = [cat.lower() for cat in CATS_TO_REMOVE]
mask = df['main_category'].str.lower().isin(cats_to_remove)
df = df[~mask]

# remove irrelevant place based by name and place_id
df = df[~df['name'].isin(NAMES_TO_DROP)]
df = df[~df['place_id'].isin(PLACE_ID_TO_DROP)]

# main_category 'Restaurant', retrieve category from categories if there are more than 1 category.
df['categories'] = df['categories'].apply(lambda x: ast.literal_eval(x))
sec_cat_mask = (df['main_category'] == 'Restaurant') & (df['categories'].apply(lambda x: x != ['Restaurant']))
df.loc[sec_cat_mask, 'main_category'] = df['categories'].apply(lambda x: x[1] if len(x) > 1 else x[0])

# recategorize remaining restaurants and remove ' restaurant' from the data
restaurant_cat_mask = df['main_category'] == 'Restaurant'
df.loc[restaurant_cat_mask, 'main_category'] = df['name'].apply(lambda x: RESTAURANT_RECATEGORIZATION.get(x, 'Restaurant'))
df['main_category'] = df['main_category'].apply(lambda x: x.replace(' restaurant', ''))

df = df.drop_duplicates(subset=['place_id', 'name', 'reviews', 'address'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4675 entries, 0 to 10093
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       4675 non-null   object 
 1   name           4675 non-null   object 
 2   reviews        4675 non-null   int64  
 3   main_category  4675 non-null   object 
 4   categories     4675 non-null   object 
 5   main_rating    4675 non-null   float64
 6   address        4675 non-null   object 
 7   link           4675 non-null   object 
 8   review_photos  4675 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 365.2+ KB


In [ ]:
df

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
0,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,"['Seafood restaurant', 'Asian restaurant', 'Di...",4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...
1,ChIJP2EDPqAZ2jERJKipSQq7qY8,Red House Seafood Nanyang (Clarke Quay),730,Seafood,['Seafood restaurant'],4.1,"3C River Valley Road 01-02/03, The Cannery, 17...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipMe...
2,ChIJSW7NnhAZ2jERj8p3W_nnhGE,Red House Seafood at Esplanade,186,Seafood,"['Seafood restaurant', 'Chinese restaurant']",4.4,"8 Raffles Ave., #01-14 / 16 Esplanade, Singapo...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipOJ...
3,ChIJNxLu-EAb2jER_a4Y4ahn8TU,Red Sparrow,496,Vietnamese,"['Vietnamese restaurant', 'Asian restaurant']",4.0,"11 Dempsey Rd, #01-18, 249673, Singapore 249673",https://www.google.com/maps/place/Red+Sparrow/...,['https://lh5.googleusercontent.com/p/AF1QipO8...
4,ChIJIds6rnUZ2jERPx6E0UCf6HM,Red Star Restaurant,3725,Cantonese,"['Cantonese restaurant', 'Chinese restaurant',...",3.7,"Block 54 Chin Swee Rd, #07-23, Singapore 160054",https://www.google.com/maps/place/Red+Star+Res...,['https://lh5.googleusercontent.com/p/AF1QipN-...
...,...,...,...,...,...,...,...,...,...
10089,ChIJA2UPtIwZ2jERE6WwKP0x278,Hua Ting Steamboat,70,Steamboat,"['Steamboat restaurant', 'Chinese restaurant']",4.2,"Mezzanine level #01-08, Claymore Connect, 442 ...",https://www.google.com/maps/place/Hua+Ting+Ste...,['https://lh5.googleusercontent.com/p/AF1QipM4...
10090,ChIJ3XkDF-EW2jERnoxsx_i5CcY,Hualong Fishhead Steamboat,173,Steamboat,"['Steamboat restaurant', 'Asian restaurant', '...",3.8,"347 Ang Mo Kio Ave 3, #01-2142 Blk 347, Singap...",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMY...
10091,ChIJLfNb-H0X2jERHknheYoXfgY,Hualong Fishhead Steamboat,45,Chinese,['Chinese restaurant'],3.8,"978 Toa Payoh N, #02-01, Singapore 319001",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMl...
10092,ChIJ6-YzAysX2jERKmeXREbO7VM,Hualong Fishhead Steamboat,8,Chinese,['Chinese restaurant'],2.6,"Blk 631 Hougang Ave 8, #01-06, Singapore 530631",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipOW...


In [ ]:
df = clean_restaurant_data(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4510 entries, 0 to 4510
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          4510 non-null   object 
 1   name              4510 non-null   object 
 2   reviews           4510 non-null   int64  
 3   main_category     4510 non-null   object 
 4   categories        4510 non-null   object 
 5   main_rating       4510 non-null   float64
 6   address           4510 non-null   object 
 7   link              4510 non-null   object 
 8   review_photos     4510 non-null   object 
 9   coordinates       4510 non-null   object 
 10  latitude          4510 non-null   float64
 11  longtitude        4510 non-null   float64
 12  full_postal_code  4510 non-null   object 
 13  postal_code       4510 non-null   object 
 14  district_code     4510 non-null   object 
 15  region            4510 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory

In [ ]:
df.main_category.unique()

array(['Seafood', 'Vietnamese', 'Cantonese', 'Indian', 'Restaurant',
       'Dim sum', 'Cafe', 'Singaporean', 'Indonesian', 'Halal',
       'Fine dining', 'Modern European', 'Asian fusion', 'European',
       'Thai', 'Wine bar', 'Nyonya', 'Chinese', 'French', 'Hot pot',
       'Italian', 'Vegetarian', 'Mediterranean', 'Meat dish', 'Pizza',
       'Korean', 'Japanese', 'Grill', 'Steak house', 'Sushi', 'Hamburger',
       'Bar', 'Traditional', 'Tapas', 'Diner', 'Mexican',
       'Chinese noodle', 'Pakistani', 'Sichuan', 'Turkish',
       'Modern French', 'Seafood donburi', 'Chicken', 'South Indian',
       'Andhra', 'Lebanese', 'Fish and seafood', 'Izakaya',
       'Authentic Japanese', 'Modern izakaya', 'Conveyor belt sushi',
       'Unagi', 'Oyster bar', 'Bistro', 'Event venue', 'Steamboat',
       'Buffet', 'Asian', 'Indian Muslim Restaurant', 'Malaysian', 'Nasi',
       'Padang', 'Coffee store', 'Ramen',
       'Cha chaan teng (Hong Kong-style cafe)', 'Tempura donburi',
       'Korea

In [ ]:
df.to_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/cleaned_all_restaurants_dataset.csv', index=False)

In [ ]:
d = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/cleaned_all_restaurants_dataset.csv', dtype={'full_postal_code': str, 'postal_code':str, 'district_code':str})
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510 entries, 0 to 4509
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          4510 non-null   object 
 1   name              4510 non-null   object 
 2   reviews           4510 non-null   int64  
 3   main_category     4510 non-null   object 
 4   categories        4510 non-null   object 
 5   main_rating       4510 non-null   float64
 6   address           4510 non-null   object 
 7   link              4510 non-null   object 
 8   review_photos     4510 non-null   object 
 9   coordinates       4510 non-null   object 
 10  latitude          4510 non-null   float64
 11  longtitude        4510 non-null   float64
 12  full_postal_code  4510 non-null   object 
 13  postal_code       4510 non-null   object 
 14  district_code     4510 non-null   object 
 15  region            4510 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory

In [ ]:
d

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,full_postal_code,postal_code,district_code,region
0,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,"['Seafood restaurant', 'Asian restaurant', 'Di...",4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...,"[1.2903687, 103.8352515]",1.290369,103.835251,169663,16,03,South
1,ChIJP2EDPqAZ2jERJKipSQq7qY8,Red House Seafood Nanyang (Clarke Quay),730,Seafood,['Seafood restaurant'],4.1,"3C River Valley Road 01-02/03, The Cannery, 17...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipMe...,"[1.29102, 103.8459532]",1.291020,103.845953,179022,17,06,City
2,ChIJSW7NnhAZ2jERj8p3W_nnhGE,Red House Seafood at Esplanade,186,Seafood,"['Seafood restaurant', 'Chinese restaurant']",4.4,"8 Raffles Ave., #01-14 / 16 Esplanade, Singapo...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipOJ...,"[1.2896585, 103.8557631]",1.289659,103.855763,039802,03,01,City
3,ChIJNxLu-EAb2jER_a4Y4ahn8TU,Red Sparrow,496,Vietnamese,"['Vietnamese restaurant', 'Asian restaurant']",4.0,"11 Dempsey Rd, #01-18, 249673, Singapore 249673",https://www.google.com/maps/place/Red+Sparrow/...,['https://lh5.googleusercontent.com/p/AF1QipO8...,"[1.3040143, 103.8100743]",1.304014,103.810074,249673,24,10,Central
4,ChIJIds6rnUZ2jERPx6E0UCf6HM,Red Star Restaurant,3725,Cantonese,"['Cantonese restaurant', 'Chinese restaurant',...",3.7,"Block 54 Chin Swee Rd, #07-23, Singapore 160054",https://www.google.com/maps/place/Red+Star+Res...,['https://lh5.googleusercontent.com/p/AF1QipN-...,"[1.2874996, 103.8411942]",1.287500,103.841194,160054,16,03,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,ChIJA2UPtIwZ2jERE6WwKP0x278,Hua Ting Steamboat,70,Steamboat,"['Steamboat restaurant', 'Chinese restaurant']",4.2,"Mezzanine level #01-08, Claymore Connect, 442 ...",https://www.google.com/maps/place/Hua+Ting+Ste...,['https://lh5.googleusercontent.com/p/AF1QipM4...,"[1.307564, 103.8287922]",1.307564,103.828792,238879,23,09,Central
4506,ChIJ3XkDF-EW2jERnoxsx_i5CcY,Hualong Fishhead Steamboat,173,Steamboat,"['Steamboat restaurant', 'Asian restaurant', '...",3.8,"347 Ang Mo Kio Ave 3, #01-2142 Blk 347, Singap...",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMY...,"[1.3673578, 103.8485856]",1.367358,103.848586,560347,56,20,North
4507,ChIJLfNb-H0X2jERHknheYoXfgY,Hualong Fishhead Steamboat,45,Chinese,['Chinese restaurant'],3.8,"978 Toa Payoh N, #02-01, Singapore 319001",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMl...,"[1.3431212, 103.8511463]",1.343121,103.851146,319001,31,12,Central
4508,ChIJ6-YzAysX2jERKmeXREbO7VM,Hualong Fishhead Steamboat,8,Chinese,['Chinese restaurant'],2.6,"Blk 631 Hougang Ave 8, #01-06, Singapore 530631",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipOW...,"[1.3707333, 103.8813722]",1.370733,103.881372,530631,53,19,North


In [ ]:
d.isnull().sum()

place_id            0
name                0
reviews             0
main_category       0
categories          0
main_rating         0
address             0
link                0
review_photos       0
coordinates         0
latitude            0
longtitude          0
full_postal_code    0
postal_code         0
district_code       0
region              0
dtype: int64

In [ ]:
d.duplicated().sum()

0

In [ ]:
d.postal_code.unique()

array(['16', '17', '03', '24', '18', '40', '05', '19', '08', '06', '07',
       '04', '15', '23', '20', '21', '38', '09', '52', '01', '39', '30',
       '56', '11', '25', '22', '53', '60', '46', '81', '27', '33', '31',
       '12', '55', '79', '67', '48', '54', '51', '82', '68', '76', '42',
       '45', '41', '80', '57', '32', '44', '65', '64', '73', '43', '78',
       '13', '29', '14', '69', '75', '36', '10', '59', '34', '26', '28',
       '58', '47', '35', '66', '77', '37', '61', '49', '62'], dtype=object)

In [ ]:
d.district_code.unique()

array(['03', '06', '01', '10', '07', '14', '02', '09', '08', '04', '18',
       '11', '20', '05', '19', '22', '16', '17', '12', '28', '23', '27',
       '15', '25', '26', '24', '13', '21'], dtype=object)

In [ ]:
d.region.unique()

array(['South', 'City', 'Central', 'East', 'North', 'West'], dtype=object)

In [ ]:
# after main_rating filter
df = clean_restaurant_data(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3827 entries, 0 to 3827
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          3827 non-null   object 
 1   name              3827 non-null   object 
 2   reviews           3827 non-null   int64  
 3   main_category     3827 non-null   object 
 4   categories        3827 non-null   object 
 5   main_rating       3827 non-null   float64
 6   address           3827 non-null   object 
 7   link              3827 non-null   object 
 8   review_photos     3827 non-null   object 
 9   coordinates       3827 non-null   object 
 10  latitude          3827 non-null   float64
 11  longtitude        3827 non-null   float64
 12  full_postal_code  3827 non-null   object 
 13  postal_code       3827 non-null   object 
 14  district_code     3827 non-null   object 
 15  region            3827 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory

In [ ]:
restaurant_df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/cleaned_all_restaurants_dataset.csv')
restaurant_df

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,full_postal_code,postal_code,district_code,region
0,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,"['Seafood restaurant', 'Asian restaurant', 'Di...",4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...,"[1.2903687, 103.8352515]",1.290369,103.835251,169663,16,3,South
1,ChIJP2EDPqAZ2jERJKipSQq7qY8,Red House Seafood Nanyang (Clarke Quay),730,Seafood,['Seafood restaurant'],4.1,"3C River Valley Road 01-02/03, The Cannery, 17...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipMe...,"[1.29102, 103.8459532]",1.291020,103.845953,179022,17,6,City
2,ChIJSW7NnhAZ2jERj8p3W_nnhGE,Red House Seafood at Esplanade,186,Seafood,"['Seafood restaurant', 'Chinese restaurant']",4.4,"8 Raffles Ave., #01-14 / 16 Esplanade, Singapo...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipOJ...,"[1.2896585, 103.8557631]",1.289659,103.855763,39802,3,1,City
3,ChIJNxLu-EAb2jER_a4Y4ahn8TU,Red Sparrow,496,Vietnamese,"['Vietnamese restaurant', 'Asian restaurant']",4.0,"11 Dempsey Rd, #01-18, 249673, Singapore 249673",https://www.google.com/maps/place/Red+Sparrow/...,['https://lh5.googleusercontent.com/p/AF1QipO8...,"[1.3040143, 103.8100743]",1.304014,103.810074,249673,24,10,Central
4,ChIJIds6rnUZ2jERPx6E0UCf6HM,Red Star Restaurant,3725,Cantonese,"['Cantonese restaurant', 'Chinese restaurant',...",3.7,"Block 54 Chin Swee Rd, #07-23, Singapore 160054",https://www.google.com/maps/place/Red+Star+Res...,['https://lh5.googleusercontent.com/p/AF1QipN-...,"[1.2874996, 103.8411942]",1.287500,103.841194,160054,16,3,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,ChIJA2UPtIwZ2jERE6WwKP0x278,Hua Ting Steamboat,70,Steamboat,"['Steamboat restaurant', 'Chinese restaurant']",4.2,"Mezzanine level #01-08, Claymore Connect, 442 ...",https://www.google.com/maps/place/Hua+Ting+Ste...,['https://lh5.googleusercontent.com/p/AF1QipM4...,"[1.307564, 103.8287922]",1.307564,103.828792,238879,23,9,Central
4506,ChIJ3XkDF-EW2jERnoxsx_i5CcY,Hualong Fishhead Steamboat,173,Steamboat,"['Steamboat restaurant', 'Asian restaurant', '...",3.8,"347 Ang Mo Kio Ave 3, #01-2142 Blk 347, Singap...",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMY...,"[1.3673578, 103.8485856]",1.367358,103.848586,560347,56,20,North
4507,ChIJLfNb-H0X2jERHknheYoXfgY,Hualong Fishhead Steamboat,45,Chinese,['Chinese restaurant'],3.8,"978 Toa Payoh N, #02-01, Singapore 319001",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMl...,"[1.3431212, 103.8511463]",1.343121,103.851146,319001,31,12,Central
4508,ChIJ6-YzAysX2jERKmeXREbO7VM,Hualong Fishhead Steamboat,8,Chinese,['Chinese restaurant'],2.6,"Blk 631 Hougang Ave 8, #01-06, Singapore 530631",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipOW...,"[1.3707333, 103.8813722]",1.370733,103.881372,530631,53,19,North


In [ ]:
restaurant_df[restaurant_df['main_category']=='Açaí shop']

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,full_postal_code,postal_code,district_code,region
2063,ChIJWbs-UOwZ2jERwP0GZgSvsgk,iNCREDi BOWL - Clarke Quay,74,Açaí shop,"['Restaurant', 'Açaí shop', 'Beer garden', 'Br...",4.4,"3A River Valley Rd, Block A #01-07 Clarke Quay...",https://www.google.com/maps/place/iNCREDi+BOWL...,['https://lh5.googleusercontent.com/p/AF1QipNz...,"[1.2899644, 103.8452462]",1.289964,103.845246,179020,17,6,City


## Reviews Dataset

In [4]:
reviews_df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/reviews_data/all_reviews_dataset.csv', low_memory=False)
reviews_df

,place_id,name,review_id,rating,review_text,published_at,published_at_date,review_likes_count,response_from_owner_text,response_from_owner_ago,response_from_owner_date,total_number_of_reviews_by_reviewer,total_number_of_photos_by_reviewer,is_local_guide,review_translated_text,response_from_owner_translated_text
0,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSUM3bE9DZ2lRRRAB,5,Delicious delicious delicious! Had the Tom yum...,2 weeks ago,2024-08-15 00:17:31.766742,0,NaN,NaN,NaN,453.0,1210.0,True,NaN,NaN
1,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChZDSUhNMG9nS0VJQ0FnSURicHQyb2Z3EAE,4,Fairly authentic Pad Thai cooked by a Thai per...,3 weeks ago,2024-08-08 00:17:31.769063,0,NaN,NaN,NaN,1381.0,8425.0,True,NaN,NaN
2,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSURieXYtTGdnRRAB,5,I had the fried kway tiao which cost $6. If ha...,3 weeks ago,2024-08-08 00:17:31.771272,0,NaN,NaN,NaN,615.0,2361.0,True,NaN,NaN
3,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChdDSUhNMG9nS0VJQ0FnSUNiOFlhSzl3RRAB,3,I got the prawn omelette rice here for $6. The...,a month ago,2024-07-29 00:17:31.773506,0,NaN,NaN,NaN,29.0,40.0,False,NaN,NaN
4,ChIJe3o1EBQZ2jERzAmaXpqhpAM,Bangkok Thai Food,ChZDSUhNMG9nS0VJQ0FnSUN6NnJhZEp3EAE,5,Really tasty Thai place in chinatown! The gree...,2 months ago,2024-06-29 00:17:31.775457,0,NaN,NaN,NaN,38.0,12.0,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148069,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSUNRZ0plZ2l3RRAB,4,Vegetarian eatery with good range of dishes th...,8 years ago,2016-08-30 15:41:51.335461,0,NaN,NaN,NaN,100.0,189.0,True,NaN,NaN
4148070,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSURBc2VmdGt3RRAB,4,Very nice vegetarian porridge and side dishes,8 years ago,2016-08-30 15:41:51.339127,0,NaN,NaN,NaN,344.0,1137.0,True,NaN,NaN
4148071,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChdDSUhNMG9nS0VJQ0FnSUNRLXN5MG1BRRAB,1,expensive and not nice!,8 years ago,2016-08-30 15:41:51.342079,2,NaN,NaN,NaN,14.0,NaN,False,NaN,NaN
4148072,ChIJ5TMoXCMY2jERzpazbjQOs6M,Rice House Vegetarian,ChZDSUhNMG9nS0VJQ0FnSURnM3FMWWN3EAE,5,Cheap and delicious,8 years ago,2016-08-30 15:41:51.345018,0,NaN,NaN,NaN,29.0,48.0,True,NaN,NaN


In [5]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4148074 entries, 0 to 4148073
Data columns (total 16 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   place_id                             object 
 1   name                                 object 
 2   review_id                            object 
 3   rating                               int64  
 4   review_text                          object 
 5   published_at                         object 
 6   published_at_date                    object 
 7   review_likes_count                   int64  
 8   response_from_owner_text             object 
 9   response_from_owner_ago              object 
 10  response_from_owner_date             object 
 11  total_number_of_reviews_by_reviewer  float64
 12  total_number_of_photos_by_reviewer   float64
 13  is_local_guide                       object 
 14  review_translated_text               object 
 15  response_from_owner_translated_t

In [ ]:
reviews_df.duplicated().sum()

74

In [ ]:
reviews_df.review_text.isnull().sum()

1478256

In [6]:
restaurant = data.copy()
review = reviews_df.copy()

In [ ]:
restaurant = reviews_df.drop(columns=['name', 'review_id'], inplace=True)

# Inner merge
merged_df = restaurants_df.merge(reviews_df, how='inner', on='place_id')

# Remove duplicates
merged_df.drop_duplicates(inplace=True)

In [7]:
merged_df = clean_data(restaurant, review)
merged_df

,place_id,name,reviews,main_category,main_rating,address,link,review_photos,cuisine,latitude,...,district_code,region,rating,review_text,published_at_date,review_likes_count,response_from_owner_text,total_number_of_reviews_by_reviewer,total_number_of_photos_by_reviewer,is_local_guide
0,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...,Seafood,1.290369,...,03,South,5,Food is Wow,2024-08-28,0,,461.0,21.0,True
1,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...,Seafood,1.290369,...,03,South,4,I always stay at the Grand Copthorne Waterfron...,2024-08-23,0,,333.0,792.0,True
2,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...,Seafood,1.290369,...,03,South,5,My family and I had a gathering at Red House S...,2024-08-16,0,,5.0,0.0,False
3,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...,Seafood,1.290369,...,03,South,5,Celebrated my MIL 70th birthday and pre-coordi...,2024-08-16,0,,3.0,14.0,False
4,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...,Seafood,1.290369,...,03,South,5,Had a wonderful dinner here tonight. The highl...,2024-07-30,0,,714.0,2924.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737156,ChIJ0T6YJPkZ2jER4SLNFf4Zthg,炭香 Charcoal Fish Head Steamboat Kallang,490,Chinese,4.1,"5 Kallang Pl, Singapore 339152",https://www.google.com/maps/place/%E7%82%AD%E9...,['https://lh5.googleusercontent.com/p/AF1QipM_...,Chinese,1.317298,...,12,Central,1,"Almost 2 hours waiting time, order taken at 6....",2023-08-29,5,,7.0,5.0,False
737157,ChIJ0T6YJPkZ2jER4SLNFf4Zthg,炭香 Charcoal Fish Head Steamboat Kallang,490,Chinese,4.1,"5 Kallang Pl, Singapore 339152",https://www.google.com/maps/place/%E7%82%AD%E9...,['https://lh5.googleusercontent.com/p/AF1QipM_...,Chinese,1.317298,...,12,Central,5,2nd time trying the fish head steamboat. Its t...,2023-08-29,0,,12.0,3.0,True
737158,ChIJ0T6YJPkZ2jER4SLNFf4Zthg,炭香 Charcoal Fish Head Steamboat Kallang,490,Chinese,4.1,"5 Kallang Pl, Singapore 339152",https://www.google.com/maps/place/%E7%82%AD%E9...,['https://lh5.googleusercontent.com/p/AF1QipM_...,Chinese,1.317298,...,12,Central,5,The food here is most excellent and the servic...,2023-08-29,1,,8.0,2.0,False
737159,ChIJ0T6YJPkZ2jER4SLNFf4Zthg,炭香 Charcoal Fish Head Steamboat Kallang,490,Chinese,4.1,"5 Kallang Pl, Singapore 339152",https://www.google.com/maps/place/%E7%82%AD%E9...,['https://lh5.googleusercontent.com/p/AF1QipM_...,Chinese,1.317298,...,12,Central,5,reserved in advance so wait wasnt long. added ...,2023-08-29,0,,23.0,8.0,True


In [8]:
merged_df.to_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/merged_dataset.csv', index=False)

In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 619630 entries, 0 to 737160
Data columns (total 23 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   place_id                             619630 non-null  object 
 1   name                                 619630 non-null  object 
 2   reviews                              619630 non-null  int64  
 3   main_category                        619630 non-null  object 
 4   main_rating                          619630 non-null  float64
 5   address                              619630 non-null  object 
 6   link                                 619630 non-null  object 
 7   review_photos                        619630 non-null  object 
 8   cuisine                              619630 non-null  object 
 9   latitude                             619630 non-null  float64
 10  longtitude                           619630 non-null  float64
 11  full_postal_c